## Optimização de hyper-parâmetros

A ideia é que esta função selecione um número X de diferentes redes.

Depois vamos criar uma função que para cada rede, vai fazer o treino e testar o score com os dados de validação. Essa função depois ordena consoante o melhor valor de AUC.

In [1]:
from sklearn.model_selection import ParameterSampler
import random as r
def selecaoHyperParametros(d,neuronios,nrCamadas,reg,n_it):
    lista_parametros = list(ParameterSampler(d, n_iter=n_it, random_state=10))
    r.seed(10)
    i = 0
    for var in lista_parametros:
        var['nrCamadas'] = r.choice(nrCamadas)
        
        var['topologia'] = r.choices(neuronios,k=var['nrCamadas'])
        while var in lista_parametros[:i]:
            var['topologia'] = r.choices(neuronios,k=var['nrCamadas'])
            
        aux = var['regularizer']
        if aux == 1:
            var['l1'] = r.choice(reg)
        elif aux == 2:
            var['l2'] = r.choice(reg)
        elif aux == 3:
            var['l1'] = r.choice(reg)
            var['l2'] = r.choice(reg)
        else:
            pass
        i+=1
    return lista_parametros

In [2]:
from keras import models,layers,regularizers
def criaRede(param,inputSize):
    model=models.Sequential()
    aux = param['regularizer']
    kernel_reg = None
    '''if aux == 1:
        kernel_reg = regularizers.l1(param['l1'])
    elif aux == 2:
        kernel_reg = regularizers.l2(param['l2'])
    elif aux == 3:
        kernel_reg = regularizers.l1_l2(l1=param['l1'],l2=param['l2'])
    else:
        pass '''
    
    model.add(layers.Dense(param['topologia'][0],activation=param['ativacao'],
                           kernel_regularizer=kernel_reg,input_shape=(inputSize,)))
    if param['dropout'] > 0:
        model.add(layers.Dropout(param['dropout']))
    for var in param['topologia'][1:]:
        model.add(layers.Dense(var,activation=param['ativacao'],kernel_regularizer=kernel_reg))
        if param['dropout'] > 0:
            model.add(layers.Dropout(param['dropout']))
    model.add(layers.Dense(1,activation='sigmoid'))
    model.compile(optimizer=param['optimizer'],
    loss='binary_crossentropy',
    metrics=['accuracy'])
    return model

Using TensorFlow backend.


In [3]:
from sklearn.metrics import roc_curve, auc
from keras.callbacks import EarlyStopping
def optimizacaoHyperParametros(d,neuronios,nrCamadas,reg,trainX,trainY,valX,valY,n_it):
    params = selecaoHyperParametros(d,neuronios,nrCamadas,reg,n_it)
    for param in params:
        print(param)
        rede = criaRede(param,trainX.shape[1])
        if param['early_stopping'] > 0:
            early = EarlyStopping(monitor='val_loss', patience=param['early_stopping'],
                                  min_delta=0, verbose=True, mode='auto')
            callb = [early]
            history = rede.fit(trainX,
                                trainY,
                                epochs=param['epochs'],
                                batch_size=param['batch_size'],
                                validation_data=(valX,valY),
                                callbacks=callb)
        else:
            history = rede.fit(trainX,
                                trainY,
                                epochs=param['epochs'],
                                batch_size=param['batch_size'],
                                validation_data=(valX,valY))
        pred = rede.predict(valX)
        false_positive_rate, true_positive_rate, thresholds = roc_curve(valY, pred)
        score = auc(false_positive_rate, true_positive_rate)
        param['score'] = score
    return params

## Aqui estão os parametros que vamos optimizar

### Faltam ainda ver o dropout, early stopping e regularização

### Optimizer temos de ver os parâmetros existentes também:
- rmsprop
    - rho: float >= 0.
    - epsilon: float >= 0. Fuzz factor. If None, defaults to K.epsilon().
    - decay: float >= 0. Learning rate decay over each update.
- SGD
    - momentum: float >= 0. Parameter that accelerates SGD in the relevant direction and dampens oscillations.
    - decay: float >= 0. Learning rate decay over each update.
    - nesterov: boolean. Whether to apply Nesterov momentum.
- Adam
    - beta_1: float, 0 < beta < 1. Generally close to 1.
    - beta_2: float, 0 < beta < 1. Generally close to 1.
    - epsilon: float >= 0. Fuzz factor. If None, defaults to K.epsilon().
    - decay: float >= 0. Learning rate decay over each update.
    - amsgrad: boolean. Whether to apply the AMSGrad variant of this algorithm from the paper "On the Convergence of Adam and Beyond"
    
##### Todos têm learning rate!
##### Existem ainda mais optimizers mas não sei se vale a pena ver todos!

In [4]:
dicionario = {
    'ativacao':['relu',#'exponential',
                'tanh','sigmoid','linear'],
    'epochs':[10,20],
    'batch_size':[64,128,256,512],
    'optimizer':['rmsprop','adam','sgd'],
    'dropout':[0.0,0.1,0.2,0.3,0.4],
    'regularizer':[0],#1,2,3],
    'early_stopping':[0,2,3,4,5]
}
neuronios = [2,3,4,5,8,10,16,32,64]
valores_l1 = [0.1,0.01,0.001] 
nrCamadas = [1,2,3,4,5,6,7,8]

In [5]:
import pandas as pd
numericos = ['AVProductsInstalled',
'AVProductsEnabled',
'Census_ProcessorCoreCount',
'Census_PrimaryDiskTotalCapacity',
'Census_SystemVolumeTotalCapacity',
'Census_TotalPhysicalRAM',
'Census_InternalPrimaryDiagonalDisplaySizeInInches',
'Census_InternalPrimaryDisplayResolutionHorizontal',
'Census_InternalPrimaryDisplayResolutionVertical',
'Census_InternalBatteryNumberOfCharges']
dtype = {}
for df in pd.read_csv('final_sembat.csv',low_memory=False,chunksize=10):
    for var in df.columns:
        if var not in numericos:
            dtype[var] = 'int8'
    break

In [6]:
import gc
del df
gc.collect()

0

In [7]:
import pandas as pd
auxPred = pd.DataFrame()
i = 0
for tp in pd.read_csv('final_sembat.csv',low_memory=False,chunksize=50000,dtype=dtype):
    if i == 0:
        auxPred = pd.concat([auxPred,tp])
    else:
        auxPred = pd.concat([auxPred,tp],ignore_index=True)
    i+=1
    print(i)

1
2
3
4
5
6
7
8
9
10
11
12


In [8]:
trainX = auxPred.loc[:499999,auxPred.columns!='HasDetections']
valX = auxPred.loc[500000:549999,auxPred.columns!='HasDetections']
trainY = auxPred.loc[:499999,'HasDetections']
valY = auxPred.loc[500000:549999,'HasDetections']

In [9]:
del auxPred
gc.collect()

14

In [10]:
trainX.shape

(500000, 707)

In [11]:
valX.shape

(50000, 707)

In [12]:
trainY.shape

(500000,)

In [13]:
valY.shape

(50000,)

In [14]:
import gc
import math
from sklearn.feature_selection import VarianceThreshold
def realizaVarThreshold():
    indices = []
    col = trainX.columns
    total = len(col)
    chunk = math.floor(total / 10)
    print(chunk)
    quantos = 0
    for i in range(chunk):
        sel = VarianceThreshold(threshold=0.001)
        try:
            sel.fit(trainX[col[quantos:quantos+10]])
            aux = [i+quantos for i in sel.get_support(indices=True)]
            indices.extend(aux)
        except:
            pass
        del sel
        gc.collect()
        quantos = quantos+10
    sel = VarianceThreshold(threshold=0.001)
    try:
        sel.fit(trainX[col[quantos:quantos+7]])
        indices.extend(sel.get_support(indices=True))
    except:
        pass
    del sel
    gc.collect()
    return indices

In [15]:
indices = realizaVarThreshold()
print(len(indices))
col = []
coln = trainX.columns
for i in indices:
       col.append(coln[i])

70
300


In [16]:
res = optimizacaoHyperParametros(dicionario,neuronios,nrCamadas,valores_l1,
                          trainX[col],trainY,
                          valX[col],valY,
                                15)

{'regularizer': 0, 'optimizer': 'sgd', 'epochs': 20, 'early_stopping': 5, 'dropout': 0.2, 'batch_size': 64, 'ativacao': 'sigmoid', 'nrCamadas': 1, 'topologia': [5]}
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Train on 500000 samples, validate on 50000 samples
Epoch 1/20
500000/500000 [==============================] - 13s 26us/step - loss: 0.6965 - acc: 0.5034 - val_loss: 0.6930 - val_acc: 0.5090
Epoch 2/20
500000/500000 [==============================] - 14s 28us/step - loss: 0.6930 - acc: 0.5075 - val_loss: 0.6930 - val_acc: 0.5090
Epoch 3/20
500000/500000 [==============================] - 15s 31us/step - loss: 0.6930 - acc: 0.5072 - val_loss: 0.6930 - val_acc: 0.5090
Epoch 4/20
500000/500000 [==============================] - 13s 27us/step - loss: 0.6930 - acc: 0.5075 - val_loss: 0.6930 - val_a

500000/500000 [==============================] - 17s 35us/step - loss: 0.6950 - acc: 0.5005 - val_loss: 0.6931 - val_acc: 0.4997
Epoch 2/10
500000/500000 [==============================] - 17s 34us/step - loss: 0.6931 - acc: 0.5049 - val_loss: 0.6930 - val_acc: 0.4997
Epoch 3/10
500000/500000 [==============================] - 17s 34us/step - loss: 0.6931 - acc: 0.5055 - val_loss: 0.6930 - val_acc: 0.5090
Epoch 4/10
500000/500000 [==============================] - 17s 34us/step - loss: 0.6930 - acc: 0.5062 - val_loss: 0.6930 - val_acc: 0.5090
Epoch 5/10
500000/500000 [==============================] - 17s 33us/step - loss: 0.6931 - acc: 0.5059 - val_loss: 0.6930 - val_acc: 0.5090
Epoch 6/10
500000/500000 [==============================] - 17s 34us/step - loss: 0.6930 - acc: 0.5064 - val_loss: 0.6930 - val_acc: 0.5090
Epoch 7/10
500000/500000 [==============================] - 17s 34us/step - loss: 0.6931 - acc: 0.5069 - val_loss: 0.6930 - val_acc: 0.5090
Epoch 8/10
500000/500000 [=====

500000/500000 [==============================] - 11s 23us/step - loss: 7.8640 - acc: 0.5101 - val_loss: 7.8899 - val_acc: 0.5090
Epoch 2/10
500000/500000 [==============================] - 10s 21us/step - loss: 7.9181 - acc: 0.5077 - val_loss: 7.8886 - val_acc: 0.5091
Epoch 3/10
500000/500000 [==============================] - 10s 21us/step - loss: 7.8790 - acc: 0.5098 - val_loss: 7.8899 - val_acc: 0.5090
Epoch 4/10
500000/500000 [==============================] - 11s 21us/step - loss: 7.9039 - acc: 0.5084 - val_loss: 7.8899 - val_acc: 0.5090
Epoch 5/10
500000/500000 [==============================] - 10s 21us/step - loss: 7.8885 - acc: 0.5093 - val_loss: 7.8899 - val_acc: 0.5090
Epoch 6/10
500000/500000 [==============================] - 11s 21us/step - loss: 7.8811 - acc: 0.5097 - val_loss: 7.8899 - val_acc: 0.5090
Epoch 7/10
500000/500000 [==============================] - 10s 21us/step - loss: 7.9022 - acc: 0.5085 - val_loss: 7.8899 - val_acc: 0.5090
Epoch 00007: early stopping
{'r

In [17]:
resultado = pd.DataFrame(res)
resultado.sort_values(by=['score'],ascending=False)

,ativacao,batch_size,dropout,early_stopping,epochs,nrCamadas,optimizer,regularizer,score,topologia
6,sigmoid,256,0.0,4,10,4,sgd,0,0.526180,"[64, 10, 2, 2]"
9,tanh,256,0.4,3,10,6,rmsprop,0,0.522535,"[32, 3, 4, 5, 2, 8]"
13,sigmoid,256,0.1,4,20,3,adam,0,0.510412,"[64, 3, 5]"
0,sigmoid,64,0.2,5,20,1,sgd,0,0.508834,[5]
10,tanh,512,0.1,3,20,2,sgd,0,0.508834,"[8, 32]"
11,linear,512,0.1,5,10,3,rmsprop,0,0.508834,"[5, 64, 64]"
5,tanh,64,0.4,3,10,5,adam,0,0.507778,"[8, 16, 10, 3, 16]"
3,tanh,512,0.4,2,20,1,adam,0,0.507653,[5]
1,relu,512,0.2,3,20,1,sgd,0,0.500000,[3]
4,linear,128,0.1,5,20,3,sgd,0,0.500000,"[10, 5, 4]"


In [18]:
(trainY.memory_usage() + valY.memory_usage() + 
 trainX.memory_usage().sum() + valX.memory_usage().sum()) / (1000*1000)

427.900328

In [19]:
res = optimizacaoHyperParametros(dicionario,neuronios,nrCamadas,valores_l1,
                          trainX[:300000],trainY[:300000],
                          valX,valY,
                                15)

{'regularizer': 0, 'optimizer': 'sgd', 'epochs': 20, 'early_stopping': 5, 'dropout': 0.2, 'batch_size': 64, 'ativacao': 'sigmoid', 'nrCamadas': 1, 'topologia': [5]}
Train on 300000 samples, validate on 50000 samples
Epoch 1/20
300000/300000 [==============================] - 20s 68us/step - loss: 0.6955 - acc: 0.4996 - val_loss: 0.6932 - val_acc: 0.5003
Epoch 2/20
300000/300000 [==============================] - 19s 65us/step - loss: 0.6931 - acc: 0.5044 - val_loss: 0.6930 - val_acc: 0.5090
Epoch 3/20
300000/300000 [==============================] - 20s 65us/step - loss: 0.6930 - acc: 0.5067 - val_loss: 0.6930 - val_acc: 0.5090
Epoch 4/20
300000/300000 [==============================] - 20s 66us/step - loss: 0.6930 - acc: 0.5074 - val_loss: 0.6930 - val_acc: 0.5090
Epoch 5/20
300000/300000 [==============================] - 20s 65us/step - loss: 0.6930 - acc: 0.5060 - val_loss: 0.6930 - val_acc: 0.5090
Epoch 6/20
300000/300000 [==============================] - 20s 65us/step - loss: 0.

300000/300000 [==============================] - 15s 52us/step - loss: 0.6930 - acc: 0.5128 - val_loss: 0.6930 - val_acc: 0.5090
Epoch 5/10
300000/300000 [==============================] - 15s 51us/step - loss: 0.6930 - acc: 0.5125 - val_loss: 0.6930 - val_acc: 0.5090
Epoch 6/10
300000/300000 [==============================] - 15s 51us/step - loss: 0.6930 - acc: 0.5144 - val_loss: 0.6930 - val_acc: 0.5090
Epoch 7/10
300000/300000 [==============================] - 15s 51us/step - loss: 0.6930 - acc: 0.5122 - val_loss: 0.6930 - val_acc: 0.5091
Epoch 8/10
300000/300000 [==============================] - 15s 51us/step - loss: 0.6930 - acc: 0.5147 - val_loss: 0.6930 - val_acc: 0.5090
Epoch 9/10
300000/300000 [==============================] - 15s 51us/step - loss: 0.6930 - acc: 0.5128 - val_loss: 0.6930 - val_acc: 0.5090
Epoch 10/10
300000/300000 [==============================] - 15s 51us/step - loss: 0.6930 - acc: 0.5099 - val_loss: 0.6930 - val_acc: 0.5087
{'regularizer': 0, 'optimizer'

300000/300000 [==============================] - 16s 53us/step - loss: 0.6941 - acc: 0.5064 - val_loss: 0.6928 - val_acc: 0.5232
Epoch 3/20
300000/300000 [==============================] - 16s 53us/step - loss: 0.6934 - acc: 0.5072 - val_loss: 0.6929 - val_acc: 0.5090
Epoch 4/20
300000/300000 [==============================] - 16s 53us/step - loss: 0.6932 - acc: 0.5082 - val_loss: 0.6927 - val_acc: 0.5090
Epoch 5/20
300000/300000 [==============================] - 16s 53us/step - loss: 0.6930 - acc: 0.5103 - val_loss: 0.6927 - val_acc: 0.5185
Epoch 6/20
300000/300000 [==============================] - 16s 53us/step - loss: 0.6931 - acc: 0.5064 - val_loss: 0.6930 - val_acc: 0.5089
Epoch 7/20
300000/300000 [==============================] - 16s 53us/step - loss: 0.6931 - acc: 0.5067 - val_loss: 0.6930 - val_acc: 0.5089
Epoch 8/20
300000/300000 [==============================] - 16s 53us/step - loss: 0.6931 - acc: 0.5075 - val_loss: 0.6930 - val_acc: 0.5094
Epoch 00008: early stopping
{'r

In [20]:
resultado = pd.DataFrame(res)
resultado.sort_values(by=['score'],ascending=False)

,ativacao,batch_size,dropout,early_stopping,epochs,nrCamadas,optimizer,regularizer,score,topologia
6,sigmoid,256,0.0,4,10,4,sgd,0,0.521379,"[64, 10, 2, 2]"
13,sigmoid,256,0.1,4,20,3,adam,0,0.510921,"[64, 3, 5]"
9,tanh,256,0.4,3,10,6,rmsprop,0,0.509630,"[32, 3, 4, 5, 2, 8]"
10,tanh,512,0.1,3,20,2,sgd,0,0.508834,"[8, 32]"
0,sigmoid,64,0.2,5,20,1,sgd,0,0.508814,[5]
3,tanh,512,0.4,2,20,1,adam,0,0.507712,[5]
5,tanh,64,0.4,3,10,5,adam,0,0.507654,"[8, 16, 10, 3, 16]"
1,relu,512,0.2,3,20,1,sgd,0,0.500000,[3]
4,linear,128,0.1,5,20,3,sgd,0,0.500000,"[10, 5, 4]"
7,linear,64,0.4,5,20,3,sgd,0,0.500000,"[3, 64, 8]"
